In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
import pandas as pd

In [ ]:
# COMMAND ----------
container_name = 'test' #replace with azure datalake container name
account_name = 'test' #replace with azure datalake storage account name



configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "{client_id}",
"fs.azure.account.oauth2.client.secret": '{app_secret_id}',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

dbutils.fs.mount(
source = f"abfss://{container_name}@{account_name}.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolypmic",
extra_configs = configs)

In [ ]:
%fs
ls "/mnt/tokyoolypmic/transformed_data/csv_files"

In [ ]:
athletes = spark.read.format('csv').option('header','true').load('/mnt/tokyoolypmic/raw_data/Athletes.csv')
Coaches = spark.read.format('csv').option('header','true').load('/mnt/tokyoolypmic/raw_data/Coaches.csv')
EntriesGender = spark.read.format('csv').option('header','true').option("inferSchema","true").load('/mnt/tokyoolypmic/raw_data/EntriesGender.csv')
Medals = spark.read.format('csv').option('header','true').option("inferSchema","true").load('/mnt/tokyoolypmic/raw_data/Medals.csv')
Teams = spark.read.format('csv').option('header','true').option("inferSchema","true").load('/mnt/tokyoolypmic/raw_data/Teams.csv')

In [ ]:
Coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
EntriesGender.display()

Discipline,Female,Male,Total
3x3 Basketball,32,32,64
Archery,64,64,128
Artistic Gymnastics,98,98,196
Artistic Swimming,105,0,105
Athletics,969,1072,2041
Badminton,86,87,173
Baseball/Softball,90,144,234
Basketball,144,144,288
Beach Volleyball,48,48,96
Boxing,102,187,289


In [ ]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
EntriesGender = EntriesGender.withColumn("Female",col("Female").cast(IntegerType()))
EntriesGender = EntriesGender.withColumn("Male",col("Male").cast(IntegerType()))
EntriesGender = EntriesGender.withColumn("Total",col("Total").cast(IntegerType()))


In [ ]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
Medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
Medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
Medals.select('Team_Country','Gold').orderBy('Gold',ascending=False).show(5)

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
+--------------------+----+
only showing top 5 rows



In [ ]:
# write files to datalake

Coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolypmic/transformed_data/coaches")
EntriesGender.repartition(1).write.mode("overwrite").option("header","true").csv("mnt/tokyoolypmic/transformed_data/entriesgender")
Medals.repartition(1).write.mode("overwrite").option("header","true").csv("mnt/tokyoolypmic/transformed_data/medals")
Teams.repartition(1).write.mode("overwrite").option("header","true").csv("mnt/tokyoolypmic/transformed_data/teams")
athletes.repartition(1).write.mode("overwrite").option("header","true").csv("mnt/tokyoolypmic/transformed_data/athletes")

In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

